A test of Kernel feature of Kaggle

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

test.csv
train.csv



In [3]:
import pandas as pd
trainDF = pd.read_csv('../input/train.csv')
testDF = pd.read_csv('../input/test.csv')

In [4]:
Y_train1 = trainDF.loc[:,'label'].values 
X_train1 = trainDF.loc[:,'pixel0':'pixel783'].values
X_test1 = testDF.loc[:,'pixel0':'pixel783'].values 
tr_n,tr_p = X_train1.shape
tt_n,tt_p = X_test1.shape
size = 28

In [5]:
import numpy as np
X_train = X_train1.reshape((tr_n,1,size,size)).astype('float32')
X_test = X_test1.reshape((tt_n,1,size,size)).astype('float32')
# Assume the max is 255
X_train = X_train / 255
X_test = X_test / 255


In [6]:
from keras.utils import np_utils
Y_train = np_utils.to_categorical(Y_train1)

num_classes = Y_train.shape[1]


print(X_train.shape)
print(Y_train.shape)

Using TensorFlow backend.


(42000, 1, 28, 28)
(42000, 10)


In [7]:
seed = 7
np.random.seed(seed)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D

In [10]:
from keras import backend as K
K.set_image_dim_ordering('th')

In [11]:
def baseline_model():
        # create model
        model = Sequential()
        # MLP model
        #model.add(Dense(num_pixels, input_dim=num_pixels, init='normal', activation='relu'))
        #model.add(Dense(num_classes, init='normal', activation='softmax'))
        # CNN model
        model.add(Convolution2D(32, 5, 5, border_mode='valid', input_shape=(1, size, size), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dense(num_classes, activation='softmax'))
        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

In [12]:
model = baseline_model()

In [ ]:
N_EPOCH = 10
from keras.wrappers.scikit_learn import KerasClassifier
estimator = KerasClassifier(build_fn=baseline_model, nb_epoch=N_EPOCH, batch_size=50, verbose=2)
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
from sklearn.model_selection import cross_val_score
results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
print("Output: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))